In [13]:
import numpy as np
import pandas as pd

train_file = "C:/Users/mirni/Documents/Проект Лизы/set1_new.tsv"
test_file = "C:/Users/mirni/Documents/Проект Лизы/set2.tsv"

train_df = pd.read_csv(train_file, sep="\t")
test_df = pd.read_csv(test_file, sep="\t")
print("=== Первые 10 строк из конечной версии файла ===")
print(df1.head(10))

=== Первые 10 строк из конечной версии файла ===
                                    pID Gender  Age  Value    City TestName  \
0  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48    2.8  Москва     THol   
1  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48    1.6  Москва      LDL   
2  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   30.7  Москва      ALT   
3  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   12.0  Москва      LPA   
4  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   44.4  Москва      ALT   
5  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   24.5  Москва      AST   
6  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   30.5  Москва      AST   
7  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48   10.8  Москва     TBil   
8  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48    9.0  Москва      LPA   
9  4A00BBCE-00D2-432D-B3BE-0002CC6D4BF4      М   48    2.1  Москва      LDL   

  TestUnits        Date  
0      mM/L  2023-04-27  
1      mM/L  2023-04-27  
2  

In [14]:
train_df.nunique()

pID          80014
Gender           2
Age            103
Value         2984
City            29
TestName         9
TestUnits        4
Date          1241
dtype: int64

Наша задача развернуть столбец TestName и выделить результаты каждого теста как отдельную фичу. Но вопрос: что будет являться индексом строки в таком случае? Ведь даже в самых первых строках нашей таблицы виден пациент, который сделал два одинаковых анализа. НО! они были сделаны в разные дни. Поэтому есть смысл делать индексом строки ID пациента и дату анализа.

In [17]:
import pandas as pd

# приводим даты к нормальному виду
train_df["ResDate"] = pd.to_datetime(train_df["Date"], dayfirst=True).dt.date

# базовые фичи на (PatientsGuid, ResDate)
base = (train_df
        .sort_values("Date")
        .groupby(["pID", "Date"], as_index=True)
        .agg({
            "Gender": "first",
            "Age": "first",
            "City": "first"
        }))

# “широкая” таблица: по одной колонке на TestName, значение = Value
vals_wide = (train_df.pivot_table(index=["pID", "Date"],
                            columns="TestName",
                            values="Value",
                            aggfunc="first"))

# объединяем
wide = (base
        .join(vals_wide)
        .reset_index())

print(wide.head(10))


C:\Users\mirni\AppData\Local\Temp\ipykernel_17404\2084886764.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  train_df["ResDate"] = pd.to_datetime(train_df["Date"], dayfirst=True).dt.date


                                    pID        Date Gender  Age         City  \
0  00004638-FF3C-4138-B20F-CCCFB20DF046  2025-04-24      Ж   49       Москва   
1  00008699-11A6-4A43-A073-D7A29B7D9ADA  2023-06-21      Ж   32  Красногорск   
2  00008699-11A6-4A43-A073-D7A29B7D9ADA  2024-01-26      Ж   33       Москва   
3  00008699-11A6-4A43-A073-D7A29B7D9ADA  2024-08-18      Ж   33       Москва   
4  0000E88C-1F93-46AE-98D0-19DE1B5EAA83  2025-05-25      Ж   41  Красногорск   
5  00020D84-9344-45C2-89CE-569603A01FC6  2024-10-17      Ж   50       Москва   
6  0003D22A-0A53-4402-9A88-7407A0327796  2023-07-13      Ж   41       Москва   
7  0003F621-73A7-40D3-801B-C9D14ACF7FEB  2024-05-11      Ж   52        Пермь   
8  00048479-A156-4CCA-B9DF-6B7BFD1598CC  2023-11-03      Ж   66  Красногорск   
9  000562FD-AB7B-43E0-8161-10FBD25C6CEC  2022-07-29      Ж   34  Красногорск   

    ALT   AST  DBil   HDL  LDL   LPA  TBil  THol  Trig  
0  38.5  23.0   NaN  1.80  4.1   9.0  11.8   6.8  1.97  
1   9

In [21]:
wide.describe()

,Age,ALT,AST,DBil,HDL,LDL,LPA,TBil,THol,Trig
count,97547.000000,55671.000000,55554.000000,14737.000000,97547.000000,97547.000000,97547.000000,35940.000000,93133.000000,97547.000000
mean,47.359837,26.664464,25.475815,2.702321,1.498923,3.409545,29.451809,13.389048,5.481470,1.338164
std,15.031503,37.448461,32.986322,4.981902,0.390692,1.039025,39.284392,8.975412,1.391696,0.964907
min,0.000000,0.900000,4.000000,0.100000,0.070000,0.100000,0.000000,2.200000,1.100000,0.140000
25%,37.000000,14.500000,18.100000,1.600000,1.220000,2.700000,6.000000,9.000000,4.500000,0.790000
50%,46.000000,20.300000,21.800000,2.100000,1.450000,3.400000,12.000000,11.600000,5.400000,1.100000
75%,57.000000,30.200000,27.000000,2.900000,1.730000,4.100000,36.000000,15.400000,6.400000,1.590000
max,105.000000,3343.400000,3391.200000,267.700000,10.260000,17.300000,398.000000,563.100000,30.200000,31.950000


In [24]:
print(f'Количество Nan у LPA {wide["LPA"].isna().sum()}')

Количество Nan у LPA 0


Перед обучением остались следующие этапы:
1. Разделение выборки на тренировочную, валидационную и тестовую: сделаем 70, 10, 20. А также сразу учтем, что один и тот же пациент в разные выборки попасть не должен
2. Дропнуть категориальные неважные фичи для тех моделей, где это требуется: город, дата анализа(но возможно потом придется сделать сезон как фичу, так как время года может влиять теоретически)
3. Заполнение пропусков у тех людей, у которых не сданы какие-то анализы в определенный день
4. One-hot-encoding пола

In [30]:
from sklearn.model_selection import GroupShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error


# ========= 1) ХЕЛПЕР: разбиение по пациентам =========
def group_train_val_test_split(df, group_col, target_col,
                               train_size=0.70, val_size=0.10, test_size=0.20,
                               random_state=42):
    assert abs(train_size + val_size + test_size - 1.0) < 1e-9, "Доли должны суммироваться в 1"
    groups = df[group_col]

    # 1) Train vs Temp
    gss1 = GroupShuffleSplit(n_splits=1, train_size=train_size, random_state=random_state)
    train_idx, temp_idx = next(gss1.split(df, groups=groups))
    df_train, df_temp = df.iloc[train_idx], df.iloc[temp_idx]

    # 2) Temp -> Val vs Test
    gss2 = GroupShuffleSplit(n_splits=1,
                             test_size=test_size/(val_size+test_size),
                             random_state=random_state)
    val_idx_rel, test_idx_rel = next(gss2.split(df_temp, groups=df_temp[group_col]))
    df_val, df_test = df_temp.iloc[val_idx_rel], df_temp.iloc[test_idx_rel]

    # 3) Разделяем на X/y
    X_train, y_train = df_train.drop(columns=[target_col]), df_train[target_col]
    X_val,   y_val   = df_val.drop(columns=[target_col]),   df_val[target_col]
    X_test,  y_test  = df_test.drop(columns=[target_col]),  df_test[target_col]

    # 4) Проверка непересекаемости групп
    g_train = set(df_train[group_col].unique())
    g_val   = set(df_val[group_col].unique())
    g_test  = set(df_test[group_col].unique())

    assert g_train.isdisjoint(g_val),  "Train и Val пересекаются по patient_id!"
    assert g_train.isdisjoint(g_test), "Train и Test пересекаются по patient_id!"
    assert g_val.isdisjoint(g_test),   "Val и Test пересекаются по patient_id!"

    return X_train, X_val, X_test, y_train, y_val, y_test





In [31]:
# ========= 2) ПОДГОТОВКА ФИЧЕЙ =========
df = wide.copy()                   # wide — твоя широкая таблица
target = "LPA"

# пол: M -> 0, Ж -> 1
df["Gender"] = df["Gender"].map({"M": 0, "Ж": 1})

drop_cols = ["Date", "City"]
df = df.drop(columns=drop_cols)
df.head()

,pID,Gender,Age,ALT,AST,DBil,HDL,LDL,LPA,TBil,THol,Trig
0,00004638-FF3C-4138-B20F-CCCFB20DF046,1.0,49,38.5,23.0,NaN,1.80,4.1,9.0,11.8,6.8,1.97
1,00008699-11A6-4A43-A073-D7A29B7D9ADA,1.0,32,9.2,15.1,NaN,1.98,2.6,8.0,NaN,4.8,0.72
2,00008699-11A6-4A43-A073-D7A29B7D9ADA,1.0,33,12.3,18.5,NaN,2.16,3.1,12.0,NaN,5.9,0.45
3,00008699-11A6-4A43-A073-D7A29B7D9ADA,1.0,33,12.6,15.7,2.2,2.22,2.8,10.0,10.2,5.4,0.63
4,0000E88C-1F93-46AE-98D0-19DE1B5EAA83,1.0,41,NaN,NaN,1.3,1.41,4.5,18.0,6.2,6.2,1.07


In [32]:
# делим на Train / Val / Test

X_train, X_val, X_test, y_train, y_val, y_test = group_train_val_test_split(
    df, group_col="pID", target_col=target,
    train_size=0.70, val_size=0.10, test_size=0.20, random_state=42
)

# после сплита удаляем PatientsGuid из признаков
for X in (X_train, X_val, X_test):
    if "pID" in X.columns:
        X.drop(columns=["pID"], inplace=True)

def rmse(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))

In [42]:
# 1) PREP: один раз считаем импутацию/скейл
from sklearn.pipeline import make_pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler

# Если KNNImputer медленный, можно быстро переключиться на SimpleImputer(strategy="median")
imp_only   = KNNImputer(n_neighbors=3, weights="uniform")
imp_scale  = make_pipeline(KNNImputer(n_neighbors=3, weights="uniform"), StandardScaler())

Xtr_imp   = imp_only.fit_transform(X_train)     # для деревьев/бустингов
Xval_imp  = imp_only.transform(X_val)
Xtr_sc_quant   = imp_scale.fit_transform(X_train)    # для линейных/MLP/KNN
Xval_sc   = imp_scale.transform(X_val)

# 2) Бейзлайн
baseline_val = np.full_like(y_val, fill_value=np.nanmean(y_train), dtype=float)
results = [("BaselineMean", rmse(y_val, baseline_val))]


In [48]:
print(f"Результаты предсказания средним: {rmse(y_val, baseline_val)}")

Результаты предсказания средним: 40.24306821367537


Посмотрим на различные корреляции между признаками и таргетом

In [47]:
X_corr = pd.DataFrame(Xtr_imp)
def calc_correlations(df, y, task="regression"):
    results = []

    for col in df.columns:
       
        x = df[col].values

        # Пирсон
        pear, _ = pearsonr(x, y)

        # Спирмен
        spear, _ = spearmanr(x, y)

        # Кендалл
        kend, _ = kendalltau(x, y)

        # Взаимная информация
        if task == "regression":
            mi = mutual_info_regression(x.reshape(-1,1), y, random_state=42)[0]
        else:
            mi = mutual_info_classif(x.reshape(-1,1), y, random_state=42)[0]

        results.append({
            "feature": col,
            "pearson": round(pear, 3),
            "spearman": round(spear, 3),
            "kendall": round(kend, 3),
            "mutual_info": round(mi, 3)
        })

    return pd.DataFrame(results).set_index("feature")

corrs = calc_correlations(X_corr, y_train, task="regression")
print(corrs)

C:\Users\mirni\AppData\Local\Temp\ipykernel_17404\2505650305.py:10: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pear, _ = pearsonr(x, y)
C:\Users\mirni\AppData\Local\Temp\ipykernel_17404\2505650305.py:13: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spear, _ = spearmanr(x, y)


         pearson  spearman  kendall  mutual_info
feature                                         
0            NaN       NaN      NaN        0.000
1          0.074     0.097    0.066        0.009
2         -0.005    -0.000   -0.000        0.003
3         -0.005     0.005    0.003        0.000
4         -0.004    -0.008   -0.005        0.002
5          0.028     0.027    0.018        0.001
6          0.038     0.060    0.042        0.006
7         -0.002     0.004    0.003        0.004
8          0.036     0.057    0.039        0.006
9         -0.018     0.006    0.004        0.003


In [55]:
# 3) МОДЕЛИ (без пайплайнов — мы уже подготовили матрицы)
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor, callback
from lightgbm import LGBMRegressor
import lightgbm as lgb

# ЛИНЕЙНЫЕ/MLP/KNN — обучаем на Xtr_sc_quant
lin = LinearRegression()
lin.fit(Xtr_sc_quant, y_train)
results.append(("LinearRegression", rmse(y_val, lin.predict(Xval_sc))))

ridge = RidgeCV(alphas=np.logspace(-3, 3, 25), cv=5)
ridge.fit(Xtr_sc_quant, y_train)
results.append(("RidgeCV", rmse(y_val, ridge.predict(Xval_sc))))

lasso = LassoCV(alphas=np.logspace(-3, 1, 20), cv=5, max_iter=10000, n_jobs=-1)
lasso.fit(Xtr_sc_quant, y_train)
results.append(("LassoCV", rmse(y_val, lasso.predict(Xval_sc))))

knn = KNeighborsRegressor(n_neighbors=15, weights="distance")
knn.fit(Xtr_sc_quant, y_train)
results.append(("KNN(k=15)", rmse(y_val, knn.predict(Xval_sc))))

mlp = MLPRegressor(hidden_layer_sizes=(128,64), activation="relu",
                   alpha=1e-4, batch_size=256, learning_rate_init=1e-3,
                   max_iter=200, random_state=42, early_stopping=True, n_iter_no_change=10)
mlp.fit(Xtr_sc_quant, y_train)
results.append(("MLP", rmse(y_val, mlp.predict(Xval_sc))))

# ДЕРЕВЬЯ/БУСТИНГИ — обучаем на Xtr_imp с ранней остановкой ОДИН РАЗ
rf = RandomForestRegressor(
    n_estimators=400, max_depth=None, min_samples_leaf=2,
    max_features="sqrt", n_jobs=-1, random_state=42
)
rf.fit(Xtr_imp, y_train)
results.append(("RandomForest", rmse(y_val, rf.predict(Xval_imp))))


In [56]:
def fit_pick_best(model_ctor, param_grid, Xtr, ytr, Xval, yval, name):
    best_model, best_rmse, best_params = None, float("inf"), None
    for params in param_grid:
        model = model_ctor(**params)
        model.fit(Xtr, ytr)
        pred = model.predict(Xval)
        score = rmse(yval, pred)
        # print(f"{name} {params} -> RMSE {score:.4f}")  # при желании включи
        if score < best_rmse:
            best_rmse, best_model, best_params = score, model, params
    # print(f"BEST {name}: {best_params} -> {best_rmse:.4f}")
    return best_model, best_rmse

# ---------- XGBoost (без ES), быстрый грубый подбор ----------
xgb_grid = [
    dict(
        n_estimators=ne,
        learning_rate=lr,
        max_depth=md,
        min_child_weight=3,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.0,
        reg_lambda=1.0,
        n_jobs=-1,
        random_state=42,
        tree_method="hist",   # быстро
        max_bin=256
    )
    for ne in (300, 600)          # маленький потолок
    for lr in (0.07, 0.10)        # быстрее с чуть бОльшим шагом
    for md in (4, 6)              # неглубокие деревья — быстрее/стабильнее
]

xgb_model, xgb_rmse = fit_pick_best(XGBRegressor, xgb_grid, Xtr_imp, y_train, Xval_imp, y_val, "XGB")
results.append(("XGBoost", xgb_rmse))

# ---------- LightGBM (без ES), быстрый грубый подбор ----------
lgb_grid = [
    dict(
        n_estimators=ne,
        learning_rate=lr,
        num_leaves=nl,          # связана с max_depth≈log2(nl)
        max_depth=-1,
        min_child_samples=20,
        subsample=0.8,
        subsample_freq=1,
        colsample_bytree=0.8,
        reg_alpha=0.0,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1,
        boosting_type="gbdt",
        max_bin=255,
        # device_type="cpu"  # если ставили GPU, можно явно указать cpu
    )
    for ne in (400, 800)
    for lr in (0.05, 0.08)
    for nl in (31, 63)
]

lgb_model, lgb_rmse = fit_pick_best(LGBMRegressor, lgb_grid, Xtr_imp, y_train, Xval_imp, y_val, "LGBM")
results.append(("LightGBM", lgb_rmse))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001891 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1936
[LightGBM] [Info] Number of data points in the train set: 68177, number of used features: 9
[LightGBM] [Info] Start training from score 29.173953


C:\Users\mirni\miniconda3\envs\ds\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001651 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1936
[LightGBM] [Info] Number of data points in the train set: 68177, number of used features: 9
[LightGBM] [Info] Start training from score 29.173953


C:\Users\mirni\miniconda3\envs\ds\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001731 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1936
[LightGBM] [Info] Number of data points in the train set: 68177, number of used features: 9
[LightGBM] [Info] Start training from score 29.173953


...

C:\Users\mirni\miniconda3\envs\ds\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001203 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1936
[LightGBM] [Info] Number of data points in the train set: 68177, number of used features: 9
[LightGBM] [Info] Start training from score 29.173953


C:\Users\mirni\miniconda3\envs\ds\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001417 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1936
[LightGBM] [Info] Number of data points in the train set: 68177, number of used features: 9
[LightGBM] [Info] Start training from score 29.173953


C:\Users\mirni\miniconda3\envs\ds\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
